<a href="https://colab.research.google.com/github/vijjipammi/pub/blob/main/face_recognition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install face_recognition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import face_recognition
image = face_recognition.load_image_file("/content/drive/MyDrive/face-reco/two_people.jpg")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,14))
plt.imshow(image)

In [ ]:
face_landmarks_list = face_recognition.face_landmarks(image)

In [ ]:
face_landmarks_list

In [ ]:
print("There are {} faces.".format(len(face_landmarks_list)))


In [ ]:
pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

In [ ]:
for face_landmarks in face_landmarks_list:

    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=1)

In [ ]:
display(pil_image)

## Create the Face Training/Face Embeddings

* Train 5-10 Images here
* Generate Known Face Encodings and
* Corresponding Face Names





In [ ]:
import face_recognition
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display

# This is an example of running face recognition on a single image
# and drawing a box around each person that was identified.

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("/content/drive/MyDrive/face-reco/obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("/content/drive/MyDrive/face-reco/biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding
]
known_face_names = [
    "Barack_Obama",
    "Joe_Biden"
]


print('Learned encoding for', len(known_face_encodings), 'images.')

In [ ]:
all_face_encodings = {}
all_face_encodings["Barack_Obama"]=obama_face_encoding
all_face_encodings["Joe_Biden"]=biden_face_encoding

In [ ]:
import pickle
with open('dataset_faces.dat', 'wb') as f:
    pickle.dump(all_face_encodings, f)

In [ ]:
with open('dataset_faces.dat', 'rb') as f:
	all_face_encodings = pickle.load(f)

# Grab the list of names and the list of encodings
known_face_names = list(all_face_encodings.keys())
known_face_encodings = np.array(list(all_face_encodings.values()))

In [ ]:
# Load an image with an unknown face
unknown_image = face_recognition.load_image_file("/content/drive/MyDrive/face-reco/two_people.jpg")

# Find all the faces and face encodings in the unknown image
face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)


In [ ]:
face_locations

In [ ]:
face_encodings

In [ ]:
  matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])

In [ ]:
matches

In [ ]:
face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)


In [ ]:
face_distances

In [ ]:

# Convert Uknown Image to PIL Format
pil_image = Image.fromarray(unknown_image)

# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)

# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

    name = "Unknown"

    # Or instead, use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    
    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

    # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


# Remove the drawing library from memory as per the Pillow docs
del draw

# Display the resulting image
display(pil_image)